Предобработка исходного датасета, чтобы с ним можно было работать.

# Импорты и библиотеки

In [4]:
!pip install -q polars langdetect tqdm


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [65]:
import polars as pl
from langdetect import detect
from datetime import datetime as dt
from tqdm.notebook import tqdm


def detect_languages(texts: list[str]):
    languages = []
    for text in tqdm(texts):
        try:
            languages.append(detect(text))
        except Exception as e:
            print(f"Error detecting language: {e}")
            languages.append("")
    return languages

# Скачиваем данные

In [50]:
!mkdir -p raw_data  
!wget -q --show-progress -O raw_data/interactions.json.gz https://mcauleylab.ucsd.edu/public_datasets/gdrive/goodreads/byGenre/goodreads_interactions_young_adult.json.gz
!wget -q --show-progress -O raw_data/books.json.gz https://mcauleylab.ucsd.edu/public_datasets/gdrive/goodreads/byGenre/goodreads_books_young_adult.json.gz


raw_data/interactio 100%[===================>]   1.71G  17.8MB/s    in 1m 42s  
raw_data/books.json 100%[===================>] 100.18M  16.6MB/s    in 7.2s    


In [92]:
%%time

data_folder = "raw_data/"

books = pl.read_ndjson(data_folder + "books.json.gz")
print("Books:")
print(books.shape)
display(books.head(5))

interactions = pl.read_ndjson(data_folder + "interactions.json.gz")
print("Interactions:")
print(interactions.shape)
display(interactions.head(5))

Books:
(93398, 29)


isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,similar_books,description,format,link,authors,publisher,num_pages,publication_day,isbn13,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
str,str,list[str],str,str,list[struct[2]],str,str,str,str,list[str],str,str,str,list[struct[2]],str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""""","""1""","[""147734""]","""US""","""""","[{""1057"",""to-read""}, {""101"",""young-adult""}, … {""2"",""vampire""}]","""B0056A00P4""","""true""","""4.04""","""B0056A00P4""","[""519546"", ""1295074"", ""21407416""]","""This is the final tale in the …","""""","""https://www.goodreads.com/book…","[{""50873"",""""}, {""232533"",""Ubersetzer""}]","""""","""""","""""","""""","""""","""""","""""","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","""12182387""","""4""","""285263""","""The Passion (Dark Visions, #3)""","""The Passion (Dark Visions, #3)"""
"""""","""2""","[""425995""]","""US""","""""","[{""1010"",""to-read""}, {""17"",""sci-fi""}, … {""1"",""owned-not-read""}]","""B006KLYIAG""","""true""","""3.80""","""B006KLYIAG""","[""13400912"", ""13327517"", … ""18460869""]","""Life should be simple for Cass…","""""","""https://www.goodreads.com/book…","[{""5395324"",""""}]","""""","""""","""""","""""","""""","""""","""""","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","""20135365""","""5""","""18450480""","""Hope's Daughter""","""Hope's Daughter"""
"""0698143760""","""17""","[""493993""]","""US""","""""","[{""1799"",""fantasy""}, {""1283"",""young-adult""}, … {""22"",""first-in-a-series""}]","""""","""true""","""3.80""","""""","[""15728807"", ""17182499"", … ""15801763""]","""Wanted by no one. Hunted by ev…","""ebook""","""https://www.goodreads.com/book…","[{""7314532"",""""}]","""Viking Children's""","""416""","""4""","""9780698143760""","""3""","""""","""2014""","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","""21401181""","""33""","""24802827""","""Half Bad (Half Life, #1)""","""Half Bad (Half Life, #1)"""
"""""","""9""","[""176160""]","""US""","""eng""","[{""7173"",""to-read""}, {""235"",""vampires""}, … {""3"",""high-school-life""}]","""B0042JSOQC""","""true""","""4.35""","""B004IYJDXY""","[""25861113"", ""7430195"", … ""10874337""]","""It all comes down to this. Vla…","""""","""https://www.goodreads.com/book…","[{""293603"",""""}]","""""","""""","""""","""""","""""","""""","""""","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","""10099492""","""152""","""10800440""","""Twelfth Grade Kills (The Chron…","""Twelfth Grade Kills (The Chron…"
"""0990662616""","""428""",[],"""US""","""eng""","[{""9481"",""to-read""}, {""196"",""young-adult""}, … {""3"",""2016-reading-list""}]","""""","""false""","""3.71""","""B00MW0MTGE""","[""20499652"", ""17934493"", … ""23281932""]","""The future world is at peace. …","""Paperback""","""https://www.goodreads.com/book…","[{""4018722"",""""}]","""Scripturient Books""","""351""","""6""","""9780990662617""","""10""","""Special Edition""","""2014""","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","""22642971""","""1525""","""42144295""","""The Body Electric""","""The Body Electric"""


Interactions:
(34919254, 10)


user_id,book_id,review_id,is_read,rating,review_text_incomplete,date_added,date_updated,read_at,started_at
str,str,str,bool,i64,str,str,str,str,str
"""8842281e1d1347389f2ab93d60773d…","""18667753""","""be53fe83a6fc83474052f84692f6e9…",false,0,"""""","""Wed Mar 29 00:12:52 -0700 2017""","""Wed Mar 29 00:12:52 -0700 2017""","""""",""""""
"""8842281e1d1347389f2ab93d60773d…","""428263""","""2030f56879ebcc307a4b9cd8c83200…",false,0,"""""","""Mon Mar 27 22:01:42 -0700 2017""","""Mon Mar 27 22:01:42 -0700 2017""","""""",""""""
"""8842281e1d1347389f2ab93d60773d…","""11387515""","""2fd3cd1acb30b099c135e358669639…",false,0,"""""","""Thu Jan 26 13:35:10 -0800 2017""","""Thu Jan 26 13:35:10 -0800 2017""","""""",""""""
"""8842281e1d1347389f2ab93d60773d…","""8684868""","""d29b8238762d70b7c2b67941bc81fb…",true,3,"""""","""Tue Dec 17 13:42:25 -0800 2013""","""Tue Dec 17 13:47:26 -0800 2013""","""Sun Dec 15 00:00:00 -0800 2013""","""Sat Dec 14 00:00:00 -0800 2013"""
"""8842281e1d1347389f2ab93d60773d…","""8423493""","""357c8c178fd0e06cff5c0256492316…",true,2,"""""","""Sun Dec 08 01:26:12 -0800 2013""","""Tue Dec 27 05:37:48 -0800 2016""","""Tue Dec 10 00:00:00 -0800 2013""",""""""


CPU times: user 2min, sys: 17 s, total: 2min 17s
Wall time: 57.2 s


# Небольшая предобработка

In [6]:
languages = detect_languages((books["title"] + " " + books["description"]).to_list())

books = (
    books
    .with_columns(
        language=pl.Series("language", languages),
        preprocessed_title=pl.col("title")
            .str.to_lowercase()
            .str.replace_all(r"[^a-z0-9 ]", ""),
        tags=pl.col("popular_shelves").list.eval(pl.element().struct.field("name"))
    )
    .filter(
        # removing definitely non-English
        pl.col("language").eq("en") & (pl.col("language_code").str.to_lowercase().str.contains("eng") | pl.col("language_code").eq(""))
    )
)

  0%|          | 0/93398 [00:00<?, ?it/s]

Error detecting language: No features in text.
Error detecting language: No features in text.
Error detecting language: No features in text.


In [94]:
# deduplication logic
books = (
    books
    .join(
        (
            books
            .select("preprocessed_title")
            .unique()
            .with_columns(
                item_id=pl.int_range(0, pl.len())
            )
        ),
        on="preprocessed_title",
        how="left"
    )
    .select("item_id", "book_id", "series", "tags", "title", "description", "url", "image_url", "authors")
)
interactions = interactions.join(
    books.select("item_id", "book_id").unique(), on="book_id", how="inner"
).drop("book_id")

books = (
    books
    .sort(pl.col("description").str.len_chars(), descending=True)
    .group_by("item_id").agg(
        pl.col("series").explode().unique(),
        pl.col("tags").explode().unique(),
        pl.col("title").first(),
        pl.col("description").first(),
        pl.col("url").first(),
        pl.col("image_url").filter(~pl.col("image_url").str.contains("nophoto")).first().fill_null(pl.col("image_url").first()),
        pl.col("authors").explode().unique()
    )
)
print("Books:")
print(books.shape)
display(books.head())

interactions = (
    interactions
    .with_columns(
        date_added=pl.col("date_added").str.strptime(pl.Datetime, "%a %b %d %H:%M:%S %z %Y").dt.replace_time_zone(None)
    )
    .group_by("user_id", "item_id").agg(
        pl.col("is_read").max(),
        pl.col("rating").max(),
        pl.col("date_added").min(),
    )
)

# filtering users with too few interactions
k_core = 2
interactions = interactions.join(
    interactions.group_by("user_id").agg(pl.col("item_id").n_unique().alias("unique_item_count"))
    .filter(pl.col("unique_item_count") > k_core).select("user_id"),
    on="user_id", how="inner"
)

# reducing overall dataset so that it weights less
interactions = interactions.sort("date_added")[-int(interactions.shape[0] * 0.6):]

print("Interactions:")
print(interactions.shape)
display(interactions.head())

Books:
(34322, 8)


item_id,series,tags,title,description,url,image_url,authors
i64,list[str],list[str],str,str,str,str,list[struct[2]]
25709,[null],"[""sports-novel"", ""books-i-need-to-get"", … ""lending-library""]","""On the Devil's Court""","""What would you give to be your…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""64925"",""""}]"
10135,[null],"[""rosie-wanabees"", ""will-not-read"", … ""out-of-print""]","""This Is Push: New Stories from…","""An anthology of new writing fr…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""213829"",""""}, {""81583"",""""}, … {""85088"",""""}]"
26915,[null],"[""family"", ""read-2010"", … ""male-protagonist""]","""The Cardturner: A Novel About …","""From Louis Sachar, New York Ti…","""https://www.goodreads.com/book…","""https://s.gr-assets.com/assets…","[{""6569"",""""}]"
18248,"[""534689""]","[""2nd-chance"", ""m-f-reads"", … ""love-story""]","""Love to Love Her (Silver State…","""When Blake and Rhiannon meet o…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""7059358"",""""}]"
22038,[null],"[""arc"", ""young-adult"", … ""currently-reading""]","""Privileged Positions""","""Privileged Positions,a trade p…","""https://www.goodreads.com/book…","""https://images.gr-assets.com/b…","[{""5363536"",""""}, {""5363535"",""""}]"


Interactions:
(15961918, 5)


user_id,item_id,is_read,rating,date_added
str,i64,bool,i64,datetime[μs]
"""90998a29029ecb1fbeedaff2b71752…",8473,false,0,2014-05-15 12:51:13
"""90998a29029ecb1fbeedaff2b71752…",20060,false,0,2014-05-15 12:51:14
"""90998a29029ecb1fbeedaff2b71752…",8354,false,0,2014-05-15 12:51:17
"""90998a29029ecb1fbeedaff2b71752…",11575,false,0,2014-05-15 12:51:19
"""90998a29029ecb1fbeedaff2b71752…",11861,false,0,2014-05-15 12:51:20


In [95]:
# sanity check
assert books.shape[0] == books["item_id"].n_unique()

# Разбиваем на обучающую и тестовую выборки

In [96]:
thresholding_dttm = dt.strptime(
    interactions["date_added"].describe().filter(pl.col("statistic") == "75%")["value"].item(), 
    "%Y-%m-%d %H:%M:%S"
)
print(thresholding_dttm)

train_interactions = interactions.filter(pl.col("date_added") < thresholding_dttm)
train_users = set(train_interactions["user_id"].to_list())
print("Train:")
print(train_interactions.shape)

test_interactions = (
    interactions
    .filter(
        (pl.col("date_added") >= thresholding_dttm) &
        (pl.col("user_id").is_in(train_users))  # keeping only warm users for simplicity
    )
    .sort("user_id", "rating", "is_read", descending=True)  # preparing test - sorting by relevance, better is higher
    .select("user_id", "item_id")  # the future is unknown...
)
print("Test:")
print(test_interactions.shape)

print(f"Test percentage: {test_interactions.shape[0] / (test_interactions.shape[0] + train_interactions.shape[0]):.2f}")

2016-12-10 02:46:45
Train:
(11971437, 5)
Test:
(3134593, 2)
Test percentage: 0.21


In [97]:
all_books = set(books["item_id"].to_list())
train_books = set(train_interactions["item_id"].to_list())
test_books = set(test_interactions["item_id"].to_list())
test_books_warm = test_books & train_books
test_books_cold = test_books - train_books

In [98]:
# sanity check
print(len(train_books - all_books))
print(len(test_books - all_books))
print(len(test_books_warm & test_books_cold))

0
0
0


In [99]:
print(f"N books in train: {len(train_books)}")
print(f"N warm books in test: {len(test_books & train_books)}")
print(f"N cold books in test: {len(test_books - train_books)}")
print(f"N potentially available books: {len(all_books)}")

N books in train: 31300
N warm books in test: 25592
N cold books in test: 1775
N potentially available books: 34322


In [100]:
test_interactions_warm = test_interactions.filter(pl.col('item_id').is_in(test_books_warm))
test_interactions_cold = test_interactions.filter(pl.col('item_id').is_in(test_books_cold))
print(f"N 'warm' interactions in test: {test_interactions_warm.shape[0]}")
print(f"N 'cold' interactions in test: {test_interactions_cold.shape[0]}")
print(f"'Cold' interactions percentage in test: {test_interactions_cold.shape[0] / (test_interactions_warm.shape[0] + test_interactions_cold.shape[0]):.2f}")

N 'warm' interactions in test: 2947221
N 'cold' interactions in test: 187372
'Cold' interactions percentage in test: 0.06


In [101]:
print(f"N users in train: {train_interactions['user_id'].n_unique()}")
print(f"N users in test: {test_interactions['user_id'].n_unique()}")

N users in train: 349719
N users in test: 185828


In [102]:
# sanity check
assert len(set(test_interactions["user_id"].to_list()) - set(train_interactions["user_id"].to_list())) == 0

# Сохраняем данные

In [103]:
!mkdir -p data

In [104]:
%%time

data_folder = "data/"

train_interactions.sort("date_added", "user_id", "item_id").write_parquet(data_folder + "train.pq")
test_interactions.sort("user_id", "item_id").write_parquet(data_folder + "test.pq")
books.sort("item_id").write_parquet(data_folder + "books.pq")

CPU times: user 7.93 s, sys: 392 ms, total: 8.33 s
Wall time: 5.51 s
